# Merge Files

### Import the required packages

In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
sns.set_style('whitegrid')
import glob
import folium
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

#import dask.dataframe as dd

 
print('imports ok')

imports ok


### prepare to merge the observations presence and absence file with the extended covariates file. 


In [119]:
obs = '../GIS_Datasets/observations_all.csv'
cov = '../Datasets/xcovariates.csv'
out = '../Datasets/joined_*.csv'


### compare number of records of presence and absence, dropping duplicates 

In [145]:
df1 = pd.read_csv(obs)
print(obs + " read")
pr = df1.loc[(df1.Charadrius != '0') & (df1.Charadrius != 0)]
print(pr.shape)
print(df1.shape)
df1 = df1.drop_duplicates(subset="SAMPLING_E", keep=False)      # drop duplicates
pr = df1.loc[(df1.Charadrius != '0') & (df1.Charadrius != 0)]
print(pr.shape)
print(df1.shape)

../GIS_Datasets/observations_all.csv read
(530, 36)
(249902, 36)
(530, 36)
(130812, 36)


C:\Users\Wayne\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### ArcGIS "Near" tool was used to derive additional features which were added to the file:
<ul>
<li>Wetlands ID and Distance
<li>Shoreline class ID and Distance
<li>Road ID and Distance, (USA and Canada)
<li>Island ID and Distance
<li>Lake polygon (NEAR_) ID and Distance
<ul>

In [146]:
df1.head()


,OBJECTID,FID_,Field1,SAMPLING_E,LOC_ID,LATITUDE,LONGITUDE,YEAR,MONTH,DAY,...,WETLANDS_DIST,SHORECLASS_ID,SHORECLASS_DIST,ROAD_ID,ROAD_DIST,ROAD_STATE,ISLAND_ID,ISLAND_DIST,NEAR_FID,NEAR_DIST
0,1,12,105,S27941860,L285222,43.659986,-79.309355,1921,9,248,...,4574.314558,17523,6.165561,294761,221.236837,GLB_ON_roads_albers,15942,1550.860035,4,191.920543
1,2,13,106,S27941937,L285222,43.659986,-79.309355,1921,8,243,...,4574.314558,17523,6.165561,294761,221.236837,GLB_ON_roads_albers,15942,1550.860035,4,191.920543
2,3,19,112,S28028957,L285222,43.659986,-79.309355,1921,8,233,...,4574.314558,17523,6.165561,294761,221.236837,GLB_ON_roads_albers,15942,1550.860035,4,191.920543
3,4,20,113,S28030509,L285222,43.659986,-79.309355,1921,9,267,...,4574.314558,17523,6.165561,294761,221.236837,GLB_ON_roads_albers,15942,1550.860035,4,191.920543
4,5,42,135,S29904854,L285222,43.659986,-79.309355,1922,5,144,...,4574.314558,17523,6.165561,294761,221.236837,GLB_ON_roads_albers,15942,1550.860035,4,191.920543


In [147]:
df1.columns

Index(['OBJECTID', 'FID_', 'Field1', 'SAMPLING_E', 'LOC_ID', 'LATITUDE',
       'LONGITUDE', 'YEAR', 'MONTH', 'DAY', 'TIME', 'COUNTRY', 'STATE_PROV',
       'COUNTY', 'COUNT_TYPE', 'EFFORT_HRS', 'EFFORT_DIS', 'EFFORT_ARE',
       'OBSERVER_I', 'NUMBER_OBS', 'GROUP_ID', 'PRIMARY_CH', 'Charadrius',
       'REEF_ID', 'REEF_DIST', 'WETLANDS_ID', 'WETLANDS_DIST', 'SHORECLASS_ID',
       'SHORECLASS_DIST', 'ROAD_ID', 'ROAD_DIST', 'ROAD_STATE', 'ISLAND_ID',
       'ISLAND_DIST', 'NEAR_FID', 'NEAR_DIST'],
      dtype='object')

### drop duplicate observations made by birds working in groups

In [148]:
# Drop records of duplicate observations made by surveyors working in groups

print('Shape of DataFrame:    ', df1.shape)
df1 = df1[df1.PRIMARY_CH != '?']
print('Shape of DataFrame:    ', df1.shape)
pr = df1.loc[(df1.Charadrius != '0') & (df1.Charadrius != 0)]
print(pr.shape)


Shape of DataFrame:     (130812, 36)
Shape of DataFrame:     (118925, 36)
(446, 36)


### drop unrequired columns

In [149]:
df1.drop(['FID_', 'Field1', 'OBSERVER_I', 'NUMBER_OBS', 'GROUP_ID', 'COUNTRY', 'STATE_PROV','COUNTY', 'PRIMARY_CH'], axis=1, inplace=True)
print(df1.shape)
df1.columns


(118925, 27)


Index(['OBJECTID', 'SAMPLING_E', 'LOC_ID', 'LATITUDE', 'LONGITUDE', 'YEAR',
       'MONTH', 'DAY', 'TIME', 'COUNT_TYPE', 'EFFORT_HRS', 'EFFORT_DIS',
       'EFFORT_ARE', 'Charadrius', 'REEF_ID', 'REEF_DIST', 'WETLANDS_ID',
       'WETLANDS_DIST', 'SHORECLASS_ID', 'SHORECLASS_DIST', 'ROAD_ID',
       'ROAD_DIST', 'ROAD_STATE', 'ISLAND_ID', 'ISLAND_DIST', 'NEAR_FID',
       'NEAR_DIST'],
      dtype='object')

### save to new file

In [150]:
obs_no_dupes = '../Datasets/observations_no_dupes.csv'
df1.to_csv(obs_no_dupes, index=False)
print("done")

done


### merge the observations with the extended covariates file using chunksize to manage memory usage.
### using the covariates file as the driver
### inner left join with observations on the left, 
### using sample ID 


In [182]:
output = "../Datasets/merged.csv"

df1 = pd.read_csv(obs_no_dupes, dtype=object)
print(obs_no_dupes + " read")

chunk = pd.DataFrame()
m = pd.DataFrame()
chunksize = 10**5
with open(output, 'a') as outfile:   # open the output file with append
        
    i = 0
    for chunk in pd.read_csv(cov, dtype=object, chunksize=chunksize):
        m = pd.merge(df1, chunk, how='inner', left_on='SAMPLING_E', right_on='SAMPLING_EVENT_ID')
        if i == 0:        
            m.to_csv(outfile, index=False)
        else:
            m.to_csv(outfile, header=None, index=False)
        i+=1
    print(cov, obs_no_dupes, " have been inner merged to ", output, " Chunks: ", i)
    
print("done")

../Datasets/observations_no_dupes.csv read
../Datasets/xcovariates.csv ../Datasets/observations_no_dupes.csv  have been inner merged to  ../Datasets/merged.csv  Chunks:  148
done


### presence data represented by 446 records

In [183]:
df1 = pd.read_csv(output, dtype=object)
print(output + " read")
df1.shape
pr = df1.loc[(df1.Charadrius != '0') & (df1.Charadrius != 0)]
print('presence observations:', pr.shape)

../Datasets/merged.csv read
presence observations: (446, 120)


In [184]:
df1.tail()

,OBJECTID,SAMPLING_E,LOC_ID_x,LATITUDE,LONGITUDE,YEAR_x,MONTH,DAY,TIME,COUNT_TYPE,...,UMD_FS_C8_1500_LPI,UMD_FS_C8_1500_PD,UMD_FS_C8_1500_PLAND,UMD_FS_C9_1500_ED,UMD_FS_C9_1500_LPI,UMD_FS_C9_1500_PD,UMD_FS_C9_1500_PLAND,UMD_LANDCOVER,UMD_WATERCOVER,YEAR_y
118920,130808,S13374172,L193567,43.15751,-87.90179,2013,3,71,8.75,P22,...,0.0476190476190476,1.1091806095652301e-07,4.76190476190476,0.0,0.0,0.0,0.0,13,2,2013
118921,130809,S13374174,L154628,42.839596,-83.428734,2013,2,55,8.0,P23,...,0.0952380952380952,3.32754182869602e-07,14.285714285714299,0.000205558992535021,0.0238095238095238,1.10918060956534e-07,2.38095238095238,4,1,2013
118922,130810,S13374176,L167933,45.09999,-87.63070999999998,2013,3,70,14.42,P21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,1,2013
118923,130811,S13374185,L2226183,41.7431396,-86.2909985,2013,3,67,7.33,P21,...,0.0408163265306122,1.9014524735404e-07,6.122448979591839,0.000176193422172867,0.0204081632653061,9.50726236770201e-08,2.04081632653061,5,1,2013
118924,130812,S13374186,L1298077,45.0982759,-87.63154509999998,2013,3,71,7.83,P21,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,1,2013


In [154]:
print(df1.shape)
df1.to_csv(output, index=False)
df1 = pd.read_csv(output, dtype=object)
print(output + " read")
pr = df1.loc[(df1.Charadrius != '0') & (df1.Charadrius != 0)]
print(pr.shape)

print(df1.shape)
print(df1.columns)

(237998, 126)
../Datasets/merged.csv read
(119519, 126)
(237998, 126)
Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'OBJECTID', 'SAMPLING_E',
       'LOC_ID_x', 'LATITUDE',
       ...
       'UMD_FS_C8_1500_LPI', 'UMD_FS_C8_1500_PD', 'UMD_FS_C8_1500_PLAND',
       'UMD_FS_C9_1500_ED', 'UMD_FS_C9_1500_LPI', 'UMD_FS_C9_1500_PD',
       'UMD_FS_C9_1500_PLAND', 'UMD_LANDCOVER', 'UMD_WATERCOVER', 'YEAR_y'],
      dtype='object', length=126)


### drop unrequired columns / clean up

In [155]:
df1.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1'], axis=1, inplace=True)
print(df1.shape)
df1.columns

(237998, 121)


Index(['Unnamed: 0.1.1.1.1.1', 'OBJECTID', 'SAMPLING_E', 'LOC_ID_x',
       'LATITUDE', 'LONGITUDE', 'YEAR_x', 'MONTH', 'DAY', 'TIME',
       ...
       'UMD_FS_C8_1500_LPI', 'UMD_FS_C8_1500_PD', 'UMD_FS_C8_1500_PLAND',
       'UMD_FS_C9_1500_ED', 'UMD_FS_C9_1500_LPI', 'UMD_FS_C9_1500_PD',
       'UMD_FS_C9_1500_PLAND', 'UMD_LANDCOVER', 'UMD_WATERCOVER', 'YEAR_y'],
      dtype='object', length=121)

In [156]:
df1.drop(['YEAR_y'], axis=1, inplace=True)
print(df1.shape)
df1.columns

(237998, 120)


Index(['Unnamed: 0.1.1.1.1.1', 'OBJECTID', 'SAMPLING_E', 'LOC_ID_x',
       'LATITUDE', 'LONGITUDE', 'YEAR_x', 'MONTH', 'DAY', 'TIME',
       ...
       'UMD_FS_C8_1500_ED', 'UMD_FS_C8_1500_LPI', 'UMD_FS_C8_1500_PD',
       'UMD_FS_C8_1500_PLAND', 'UMD_FS_C9_1500_ED', 'UMD_FS_C9_1500_LPI',
       'UMD_FS_C9_1500_PD', 'UMD_FS_C9_1500_PLAND', 'UMD_LANDCOVER',
       'UMD_WATERCOVER'],
      dtype='object', length=120)

### merge current working file on the left with islands dataset using island ID and object ID
### using chunksize, dropping the ID field from the islands file

In [42]:
infile = "../Datasets/merged.csv"
islands = "../GIS_Datasets/islands.csv"
output = "../Datasets/merged1.csv"

df2 = pd.read_csv(islands, dtype=object)

chunk = pd.DataFrame()
m = pd.DataFrame()
chunksize = 10**5

with open(output, 'a') as outfile:  # open new merging file with append
        
    i = 0
    for chunk in pd.read_csv(infile, dtype=object, chunksize=chunksize):
        m = pd.merge(chunk, df2, how='left', left_on='ISLAND_ID', right_on='OBJECTID_1', suffixes=['_X','_ISLANDS']).drop('OBJECTID_1', axis=1)
        if i == 0:        
            m.to_csv(outfile, index=False)
        else:
            m.to_csv(outfile, header=None, index=False)  # skip headers after the first read
        i+=1
    print(infile, islands, " have been inner merged to ", output, " Chunks: ", i)
    
print("done")




../Datasets/merged.csv ../GIS_Datasets/islands.csv  have been inner merged to  ../Datasets/merged1.csv  Chunks:  2
done


### Shape_Length and Shape_Area for islands columns added

In [43]:
m.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1,OBJECTID,SAMPLING_E,LOC_ID_x,LATITUDE,...,UMD_FS_C8_1500_PLAND,UMD_FS_C9_1500_ED,UMD_FS_C9_1500_LPI,UMD_FS_C9_1500_PD,UMD_FS_C9_1500_PLAND,UMD_LANDCOVER,UMD_WATERCOVER,YEAR_y,Shape_Length,Shape_Area
18920,118920,7815,7815,7815,7815,7815,130808,S13374172,L193567,43.157509999999995,...,4.76190476190476,0.0,0.0,0.0,0.0,13,2,2013,390.167215393130900,2998.386179074208900
18921,118921,7816,7816,7816,7816,7816,130809,S13374174,L154628,42.839596,...,14.285714285714299,0.000205558992535021,0.0238095238095238,1.10918060956534e-07,2.38095238095238,4,1,2013,6341.461034427105900,496933.728748467050000
18922,118922,7817,7817,7817,7817,7817,130810,S13374176,L167933,45.099990000000005,...,0.0,0.0,0.0,0.0,0.0,13,1,2013,3110.219725172519200,337310.750945429550000
18923,118923,7821,7821,7821,7821,7821,130811,S13374185,L2226183,41.7431396,...,6.122448979591839,0.000176193422172867,0.0204081632653061,9.50726236770201e-08,2.04081632653061,5,1,2013,114.360583445322520,813.552365732133920
18924,118924,7822,7822,7822,7822,7822,130812,S13374186,L1298077,45.0982759,...,0.0,0.0,0.0,0.0,0.0,13,1,2013,3110.219725172519200,337310.750945429550000


### prepare to merge reefs file

In [59]:
merging = "../GIS_Datasets/reefs.csv"
df2 = pd.read_csv(merging, dtype=object)
df2.columns


Index(['OBJECTID', 'Feature_Name', 'GNIS_ID', 'Description', 'County', 'State',
       'LAT', 'LONG', 'Elev_ft', 'Map__', 'Entry_Date'],
      dtype='object')

### drop unrequired fields

In [60]:

df2.drop(['Feature_Name', 'GNIS_ID','County', 'State', 'LAT', 'LONG', 'Elev_ft', 'Map__', 'Entry_Date'], axis=1, inplace=True)
print(df1.shape)
df2.columns

(390, 2)


Index(['OBJECTID', 'Description'], dtype='object')

### merge reefs file
### reefs Descriptions field now added to the working file

In [61]:
infile = "../Datasets/merged1.csv"
merging = "../GIS_Datasets/reefs.csv"
output = "../Datasets/merged2.csv"
left_to_right = "REEF_ID"
right_to_left = "OBJECTID"
suffix = "_REEF"


m = pd.DataFrame()
chunksize = 10**5

with open(output, 'a') as outfile:
        
    i = 0
    for chunk in pd.read_csv(infile, dtype=object, chunksize=chunksize):
        m = pd.merge(chunk, df2, how='left', left_on=left_to_right, right_on=right_to_left, suffixes=['_X', suffix]).drop(right_to_left, axis=1)
        if i == 0:        
            m.to_csv(outfile, index=False)
        else:
            m.to_csv(outfile, header=None, index=False)
        i+=1
    print(infile, merging, " have been inner merged to ", output, " Chunks: ", i)
    
print("done")
m.head()


../Datasets/merged1.csv ../GIS_Datasets/reefs.csv  have been inner merged to  ../Datasets/merged2.csv  Chunks:  2
done


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,OBJECTID_X,SAMPLING_E,LOC_ID_x,LATITUDE,...,UMD_FS_C9_1500_LPI,UMD_FS_C9_1500_PD,UMD_FS_C9_1500_PLAND,UMD_LANDCOVER,UMD_WATERCOVER,YEAR_y,Shape_Length,Shape_Area,OBJECTID_REEF,Description
0,100000,2326,2326,2326,2326,2326,107759,S11688993,L152740,41.7857131,...,0.0,0.0,0.0,13,2,2012,1505.291679900385800,68588.176795336942000,4,Bar
1,100001,2338,2338,2338,2338,2338,107760,S11689031,L10,43.842946000000005,...,0.0204081632653061,9.50726236771293e-08,2.04081632653061,5,1,2012,44.675717329372361,139.980207802104870,152,Bar
2,100002,2355,2355,2355,2355,2355,107761,S11689077,L190873,42.4877956,...,0.0238095238095238,1.10918060956651e-07,2.38095238095238,8,2,2012,81.190402641637760,397.267090523547100,190,Bar
3,100003,2359,2359,2359,2359,2359,107762,S11689082,L351831,41.7960139,...,0.0,0.0,0.0,0,3,2012,102.433292408201380,705.406916452271840,5,Bar
4,100004,2364,2364,2364,2364,2364,107763,S11689089,L207765,42.846250899999994,...,0.0238095238095238,1.10918060956651e-07,2.38095238095238,1,3,2012,6585.027973825160200,669922.803974343460000,114,Bar


### prepare to merge Shoreline Classes file

In [66]:
merging = "../GIS_Datasets/shoreline_classes.csv"
df2 = pd.read_csv(merging, dtype=object)
df2.columns

Index(['OBJECTID_1', 'shoreline_classes', 'GLAHF_simplified_shoreline_classes',
       'Shape_Length', 'SHORECLASS', 'LAT_Start', 'LAT_Mid', 'LAT_End',
       'LONG_Start', 'LONG_Mid', 'LONG_End'],
      dtype='object')

### drop unrequired fields 

In [67]:
df2.drop(['shoreline_classes', 'GLAHF_simplified_shoreline_classes', 'LAT_Start', 'LAT_Mid', 'LAT_End',
       'LONG_Start', 'LONG_Mid', 'LONG_End'], axis=1, inplace=True)
print(df2.shape)
df2.columns

(47693, 3)


Index(['OBJECTID_1', 'Shape_Length', 'SHORECLASS'], dtype='object')

### merge Shoreline file with the working file
### 'Shape_Length_SHORE', 'SHORECLASS' fields added

In [68]:
infile = "../Datasets/merged2.csv"
# merging = "../GIS_Datasets/reefs.csv"
output = "../Datasets/merged3.csv"
left_to_right = "SHORECLASS_ID"
right_to_left = "OBJECTID_1"
suffix = "_SHORE"

# df2 = pd.read_csv(merging, dtype=object)

m = pd.DataFrame()
chunksize = 10**5

with open(output, 'a') as outfile:
        
    i = 0
    for chunk in pd.read_csv(infile, dtype=object, chunksize=chunksize):
        m = pd.merge(chunk, df2, how='left', left_on=left_to_right, right_on=right_to_left, suffixes=['_X', suffix]).drop(right_to_left, axis=1)
        if i == 0:        
            m.to_csv(outfile, index=False)
        else:
            m.to_csv(outfile, header=None, index=False)
        i+=1
    print(infile, merging, " have been inner merged to ", output, " Chunks: ", i)
    
print("done")
m.head()


../Datasets/merged2.csv ../GIS_Datasets/shoreline_classes.csv  have been inner merged to  ../Datasets/merged3.csv  Chunks:  2
done


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,OBJECTID_X,SAMPLING_E,LOC_ID_x,LATITUDE,...,UMD_FS_C9_1500_PLAND,UMD_LANDCOVER,UMD_WATERCOVER,YEAR_y,Shape_Length_X,Shape_Area,OBJECTID_REEF,Description,Shape_Length_SHORE,SHORECLASS
0,100000,2326,2326,2326,2326,2326,107759,S11688993,L152740,41.7857131,...,0.0,13,2,2012,1505.291679900385800,68588.176795336942000,4,Bar,1002.791856499687800,1
1,100001,2338,2338,2338,2338,2338,107760,S11689031,L10,43.842946000000005,...,2.04081632653061,5,1,2012,44.675717329372361,139.980207802104870,152,Bar,692.491636010253840,2
2,100002,2355,2355,2355,2355,2355,107761,S11689077,L190873,42.4877956,...,2.38095238095238,8,2,2012,81.190402641637760,397.267090523547100,190,Bar,1002.038074371138000,1
3,100003,2359,2359,2359,2359,2359,107762,S11689082,L351831,41.7960139,...,0.0,0,3,2012,102.433292408201380,705.406916452271840,5,Bar,1000.327548639702100,1
4,100004,2364,2364,2364,2364,2364,107763,S11689089,L207765,42.846250899999994,...,2.38095238095238,1,3,2012,6585.027973825160200,669922.803974343460000,114,Bar,7914.590808276670900,1


### prepare to merge the Lakes polygon file

In [69]:
merging = "../GIS_Datasets/lakes.csv"
df2 = pd.read_csv(merging, dtype=object)
df2.columns

Index(['OBJECTID_1', 'GNIS_Name', 'AreaSqKm', 'Shape_Length', 'Shape_Area'], dtype='object')

In [70]:
df2.drop(['AreaSqKm', 'Shape_Length', 'Shape_Area'], axis=1, inplace=True)
print(df2.shape)
df2.columns

(5, 2)


Index(['OBJECTID_1', 'GNIS_Name'], dtype='object')

### merge Lakes file with the working file

In [72]:
infile = "../Datasets/merged3.csv"
merging = "../GIS_Datasets/lakes.csv"
output = "../Datasets/merged4.csv"
left_to_right = "NEAR_FID"
right_to_left = "OBJECTID_1"
suffix = "_LAKE"

# df2 = pd.read_csv(merging, dtype=object)

chunk = pd.DataFrame()
m = pd.DataFrame()
chunksize = 10**5

with open(output, 'a') as outfile:
        
    i = 0
    for chunk in pd.read_csv(infile, dtype=object, chunksize=chunksize):
        m = pd.merge(chunk, df2, how='left', left_on=left_to_right, right_on=right_to_left, suffixes=['_X', suffix]).drop(right_to_left, axis=1)
        if i == 0:        
            m.to_csv(outfile, index=False)
        else:
            m.to_csv(outfile, header=None, index=False)
        i+=1
    print(infile, merging, " have been inner merged to ", output, " Chunks: ", i)
    
print("done")
m.tail()

../Datasets/merged3.csv ../GIS_Datasets/lakes.csv  have been inner merged to  ../Datasets/merged4.csv  Chunks:  2
done


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,OBJECTID_X,SAMPLING_E,LOC_ID_x,LATITUDE,...,UMD_LANDCOVER,UMD_WATERCOVER,YEAR_y,Shape_Length_X,Shape_Area,OBJECTID_REEF,Description,Shape_Length_SHORE,SHORECLASS,GNIS_Name
18920,118920,7815,7815,7815,7815,7815,130808,S13374172,L193567,43.157509999999995,...,13,2,2013,390.167215393130900,2998.386179074208900,227,,1003.435338280289000,6,Lake Michigan
18921,118921,7816,7816,7816,7816,7816,130809,S13374174,L154628,42.839596,...,4,1,2013,6341.461034427105900,496933.728748467050000,79,Bar,1938.728787684923400,1,Lake Erie
18922,118922,7817,7817,7817,7817,7817,130810,S13374176,L167933,45.099990000000005,...,13,1,2013,3110.219725172519200,337310.750945429550000,203,Bar,1000.431228862307400,1,Lake Michigan
18923,118923,7821,7821,7821,7821,7821,130811,S13374185,L2226183,41.7431396,...,5,1,2013,114.360583445322520,813.552365732133920,8,Bar,1003.179913007360800,6,Lake Michigan
18924,118924,7822,7822,7822,7822,7822,130812,S13374186,L1298077,45.0982759,...,13,1,2013,3110.219725172519200,337310.750945429550000,203,Bar,1000.538460187597500,1,Lake Michigan


### merging the wetlands files with the working file

In [76]:
merging = "../GIS_Datasets/wetlands.csv"
df2 = pd.read_csv(merging, dtype=object)
df2.columns
       

Index(['OBJECTID_1', 'OBJECTID', 'AREA', 'PERIMETER', 'GLCWC_CWI_',
       'GLCWC_CWI1', 'USGS_QUAD', 'WETLAND_NA', 'COMMENTS', 'X_CENTROID',
       'Y_CENTROID', 'HECTARES', 'NWI', 'WWI', 'OWI', 'HWI', 'LOWI', 'H_NUM',
       'DITCH_CONS', 'DYKES', 'DAMS', 'DREDGING', 'JETTY', 'ROAD_CONST',
       'FILLED', 'MARINA', 'WASTE_SEWA', 'GEO_ID', 'COUNTRY', 'OTHER',
       'LK_BASIN', 'HGM_CLS1', 'HGM_CLS2', 'Shape_Leng', 'HGMCLS1_S',
       'Shape_Length', 'Shape_Area'],
      dtype='object')

### Drop unrequired fields leaving:
'AREA', 'PERIMETER', 'NWI', 'WWI', 'OWI', 'HWI', 'LOWI',
       'H_NUM', 'DITCH_CONS', 'DYKES', 'DAMS', 'DREDGING', 'JETTY',
       'ROAD_CONST', 'FILLED', 'MARINA', 'WASTE_SEWA', 'HGM_CLS1'

In [77]:
df2.drop(['OBJECTID', 'GLCWC_CWI_', 'GLCWC_CWI1', 'USGS_QUAD', 'WETLAND_NA', 'COMMENTS', 'X_CENTROID',
       'Y_CENTROID', 'HECTARES', 'GEO_ID', 'COUNTRY', 'OTHER', 'LK_BASIN', 'Shape_Leng', 'HGMCLS1_S',
       'Shape_Length', 'Shape_Area', 'HGM_CLS2'], axis=1, inplace=True)
print(df2.shape)
df2.columns

(8151, 19)


Index(['OBJECTID_1', 'AREA', 'PERIMETER', 'NWI', 'WWI', 'OWI', 'HWI', 'LOWI',
       'H_NUM', 'DITCH_CONS', 'DYKES', 'DAMS', 'DREDGING', 'JETTY',
       'ROAD_CONST', 'FILLED', 'MARINA', 'WASTE_SEWA', 'HGM_CLS1'],
      dtype='object')

In [78]:
infile = "../Datasets/merged4.csv"
merging = "../GIS_Datasets/lakes.csv"
output = "../Datasets/merged5.csv"
left_to_right = "WETLANDS_ID"
right_to_left = "OBJECTID_1"
suffix = "_WETLANDS"

# df2 = pd.read_csv(merging, dtype=object)

chunk = pd.DataFrame()
m = pd.DataFrame()
chunksize = 10**5

with open(output, 'a') as outfile:
        
    i = 0
    for chunk in pd.read_csv(infile, dtype=object, chunksize=chunksize):
        m = pd.merge(chunk, df2, how='left', left_on=left_to_right, right_on=right_to_left, suffixes=['_X', suffix]).drop(right_to_left, axis=1)
        if i == 0:        
            m.to_csv(outfile, index=False)
        else:
            m.to_csv(outfile, header=None, index=False)
        i+=1
    print(infile, merging, " have been inner merged to ", output, " Chunks: ", i)
    
print("done")
m.tail()

../Datasets/merged4.csv ../GIS_Datasets/lakes.csv  have been inner merged to  ../Datasets/merged5.csv  Chunks:  2
done


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,OBJECTID_X,SAMPLING_E,LOC_ID_x,LATITUDE,...,DITCH_CONS,DYKES,DAMS,DREDGING,JETTY,ROAD_CONST,FILLED,MARINA,WASTE_SEWA,HGM_CLS1
18920,118920,7815,7815,7815,7815,7815,130808,S13374172,L193567,43.157509999999995,...,T,F,F,F,F,T,T,F,T,Barrier-Protected Ridge and Swale Complex
18921,118921,7816,7816,7816,7816,7816,130809,S13374174,L154628,42.839596,...,T,F,F,T,T,T,T,T,F,"Lacustrine, Open Shoreline"
18922,118922,7817,7817,7817,7817,7817,130810,S13374176,L167933,45.099990000000005,...,F,F,F,F,F,T,F,F,F,Barrier-Protected Ridge and Swale Complex
18923,118923,7821,7821,7821,7821,7821,130811,S13374185,L2226183,41.7431396,...,F,F,F,F,F,T,T,F,F,Barrier_Protected Successional Barrier Beach L...
18924,118924,7822,7822,7822,7822,7822,130812,S13374186,L1298077,45.0982759,...,F,F,F,F,F,T,F,F,F,Barrier-Protected Ridge and Swale Complex


In [80]:
m.columns


Index(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1',
       'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1', 'OBJECTID_X',
       'SAMPLING_E', 'LOC_ID_x', 'LATITUDE',
       ...
       'DITCH_CONS', 'DYKES', 'DAMS', 'DREDGING', 'JETTY', 'ROAD_CONST',
       'FILLED', 'MARINA', 'WASTE_SEWA', 'HGM_CLS1'],
      dtype='object', length=151)

### drop unrequired fields and clean up

In [14]:
infile = "../Datasets/merged5.csv"
outfile = "../Datasets/working.csv"
df1 = pd.read_csv(infile, dtype=object)
# df1.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'Unnamed: 0.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1'], axis=1, inplace=True)


In [15]:
#df1["Charadrius"] = df1["Charadrius"].replace('0', 0)
df1.loc[(df1["Charadrius"] == '0'), ['Charadrius']] = 0
df1.loc[(df1["Charadrius"] != 0) & (df1["Charadrius"] != '0'), ['Charadrius']] = 1
 
POS_LOC_ID = [row['LOC_ID_x'] for index, row in df1[df1['Charadrius'] == 1].iterrows()]
NEG_LOC_ID = [row['LOC_ID_x'] for index, row in df1[df1['Charadrius'] == 0].iterrows()]
 
pr = df1.loc[df1.Charadrius == 1]
print('presence observations:', pr.shape)
nopr = df1.loc[df1.Charadrius == 0]
print('absence observations:', nopr.shape)

for i in POS_LOC_ID:
    df1.loc[(df1["Charadrius"] == 0) & (df1["LOC_ID_x"] == i), ['Charadrius']] = 9999
        
df1 = df1.loc[df1["Charadrius"]!=9999]


pr = df1.loc[df1.Charadrius == 1]
print('presence observations:', pr.shape)
nopr = df1.loc[df1.Charadrius == 0]
print('absence observations:', nopr.shape)


presence observations: (446, 145)
absence observations: (118479, 145)
presence observations: (446, 145)
absence observations: (96162, 145)


In [16]:
print("absence:",sum(df1["Charadrius"] == 0))
print("presence:",sum(df1["Charadrius"] == 1))
df1.Charadrius.unique()

absence: 96162
presence: 446


array([0, 1], dtype=object)

### save file

In [17]:
outfile1 = "../Datasets/merged5a.csv"
df1.to_csv(outfile1, index=False)

In [18]:
df1.drop(['LOC_ID_x','YEAR_x','OBJECTID_X', 'SAMPLING_E', 'LATITUDE', 'LONGITUDE', 'MONTH', 'TIME', 'DAY', 'COUNT_TYPE', 'EFFORT_HRS', 'EFFORT_DIS', 'EFFORT_ARE', 'REEF_ID', 'WETLANDS_ID', 'SHORECLASS_ID', 'ROAD_ID','ROAD_STATE','ISLAND_ID','NEAR_FID','Unnamed: 0.1.1.1.1.1.1', 'SAMPLING_EVENT_ID', 'LOC_ID_y','YEAR_y','OBJECTID_REEF'], axis=1, inplace=True)
pr = df1.loc[(df1.Charadrius != '0') & (df1.Charadrius != 0)]
print('presence observations:', pr.shape)
print(df1.shape)
for i in df1.columns:
    print(i)

presence observations: (446, 120)
(96608, 120)
Charadrius
REEF_DIST
WETLANDS_DIST
SHORECLASS_DIST
ROAD_DIST
ISLAND_DIST
NEAR_DIST
EASTNESS
ELEV
MODISWATER_FS_C0_1500_ED
MODISWATER_FS_C0_1500_LPI
MODISWATER_FS_C0_1500_PD
MODISWATER_FS_C0_1500_PLAND
MODISWATER_FS_C1_1500_ED
MODISWATER_FS_C1_1500_LPI
MODISWATER_FS_C1_1500_PD
MODISWATER_FS_C1_1500_PLAND
MODISWATER_FS_C2_1500_ED
MODISWATER_FS_C2_1500_LPI
MODISWATER_FS_C2_1500_PD
MODISWATER_FS_C2_1500_PLAND
MODISWATER_FS_C3_1500_ED
MODISWATER_FS_C3_1500_LPI
MODISWATER_FS_C3_1500_PD
MODISWATER_FS_C3_1500_PLAND
MODISWATER_FS_C5_1500_ED
MODISWATER_FS_C5_1500_LPI
MODISWATER_FS_C5_1500_PD
MODISWATER_FS_C5_1500_PLAND
MODISWATER_FS_C6_1500_ED
MODISWATER_FS_C6_1500_LPI
MODISWATER_FS_C6_1500_PD
MODISWATER_FS_C6_1500_PLAND
MODISWATER_FS_C7_1500_ED
MODISWATER_FS_C7_1500_LPI
MODISWATER_FS_C7_1500_PD
MODISWATER_FS_C7_1500_PLAND
NORTHNESS
UMD_FS_C0_1500_ED
UMD_FS_C0_1500_LPI
UMD_FS_C0_1500_PD
UMD_FS_C0_1500_PLAND
UMD_FS_C1_1500_ED
UMD_FS_C1_1500_LPI
UMD_F

In [19]:
df1.to_csv(outfile, index=False)